# needed

# database with college football teams, colors, logos, game info (https://collegefootballdata.com/exporter)

# open ai key so I can use their image generation (https://www.geeksforgeeks.org/generate-images-with-openai-in-python/)

# s3 so I can save designs

 


In [8]:

from PIL import Image, ImageOps, ImageDraw, ImageFont
import warnings
import requests
import pandas as pd
import json
import datetime as dt
import cfbd
import helpers as hf
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import yaml
import random
import base64


In [2]:
def monday_run(test=False):
    base_config,post_config = hf.get_config()
    cfbd_loader = hf.cfbp_handler(base_config['cfbd_api'])
    s3_mover = hf.s3_mover()
    yr = dt.date.today().year

    # refresh teams if new season, could also add something to check if any teams exist
    if hf.new_football_season():
        teams = cfbd_loader.get_team_info()
        s3_mover.pd_to_s3(teams,f'teams/all_teams_{yr}')
    else:
        # get my df of teams
        teams = s3_mover.s3_to_pd(f'teams/all_teams_{yr}')
    
    # get new schedule info and return list of games i need to do
    to_do_new,to_do_upcoming,all_games = cfbd_loader.get_schedule(yr)

    print(f'New games to do {len(to_do_new)}')
    print(f'Upcoming games to do {len(to_do_upcoming)}')

    # have to line image path for config up with
    print('new games')
    for game in to_do_new:
        print(f'    generating game {game} image')
        game_config = hf.parse_game(game,teams)
        game_config['path'] = rf"images/pre_{game}.png"
        game_config['game_id'] = {game}
        config = hf.combine_configs(base_config,game_config)
        design = hf.build_design(config,test=False)

        # do I need to save all my images to s3?
        s3_mover.image_to_s3(design,rf"post/{game_config['home_team']['shortn']}_v_{game_config['away_team']['shortn']}_{game_id}.csv")

        if not test:
            post_config['image'] = design
            hf.post(post_config)
    print('upcoming games')
    for game_id in to_do_upcoming:
        game = all_games.loc[all_games.id==game_id].to_dict('list')
        for k in game.keys():
            game[k] = game[k][0]
        print(f'    generating game {game_id} image')
        game_config = hf.parse_game(game,teams)
        game_config['game_id'] = game_id
        config = hf.combine_configs(base_config,game_config)
        design = hf.build_design(config,test=False)
        
        # do I need to save all my images to s3?
        s3_mover.image_to_s3(design,rf"pre/{game_config['home_team']['shortn']}_v_{game_config['away_team']['shortn']}_{game_id}.csv")

        if not test:
            post_config['image'] = design
            hf.post(post_config)

In [5]:
# monday_run(test=True)

- handle nan in teams table
- font auto sizing

In [4]:
#automate adding shirts to printfy/shopify

In [67]:

def post(post_dict,publish=False):
    upload_url = f"{post_dict['base_url']}/uploads/images.json"
    product_url = f"{post_dict['base_url']}/shops/{post_dict['shop_id']}/products.json"

    headers = {
    "Authorization": f"Bearer {post_dict['printify_access']}",
    "Content-Type": "application/json"
    }

    # this posts a design, but doesnt publish it
    img = post_dict['design']

    # Upload the image to the Printify media library
    data = {
    "file_name": post_dict['title'],
    "contents": img
    }
    response = requests.post(upload_url, headers=headers, json=data)
    if response.status_code == 200:
        print('image sent to printify')
        image_id = response.json()["id"]
        variants = [{"id":id,"price":post_dict['price'],"is_enabled": True} for id in post_dict['variant_ids']]

        data = {
            "title": post_dict['title'],
            "description": post_dict['description'],
            "tags": post_dict['tags'].split(', '),  # Assuming tags are comma-separated in the CSV
            "blueprint_id": post_dict['blueprint_id'],
            "print_provider_id": post_dict['print_provider_id'],
            "variants": variants,
            "print_areas": [
                {
                    "variant_ids": post_dict['variant_ids'],  # Replace with the actual variant ID
                    "placeholders": [
                        {
                            "position": "back", # post to back of shirt, put a lil one on front with just date team score
                            "images": [
                                {
                                    "id": image_id,
                                    "x": 0.5,
                                    "y": 0.5,
                                    "scale": 1,
                                    "angle": 0
                                }
                            ]
                        }
                    ]
                }
            ]
        }

        response1 = requests.post(product_url, headers=headers, json=data)
        if response1.status_code == 200:
            print("Product posted successfully in Printify")
        else:
            print("Failed to post product in Printify")

        if publish:
            # this part does the publishing
            printify_publish = f"{post_dict['base_url']}/shops/{post_dict['shop_id']}/products/{json.loads(response1.text)['id']}/publish.json"

            update_data = {
                "title": True,
                "description": True,
                "images": True,
                "variants": True,
                "tags": True,
                "keyFeatures": True,
                "shipping_template": True
            }

            # Patch request to update the product status
            response2 = requests.post(printify_publish, headers=headers, json=update_data)
            if response2.status_code == 200:
                print("Product published successfully in Printify")
            else:
                print("Failed to publish product in Printify")
    else:
        print('unable to send image to printify')

In [68]:
post(shop_config,publish=True)

image sent to printify
Product posted successfully in Printify
Product published successfully in Printify


In [69]:
def get_items(created, post_dict):
    '''
    get items, so we can filter by name to get shopify id to add to collection
    created is a list of all the things we just made to add to the list
    '''
    shopify_headers ={
    "X-Shopify-Access-Token": post_dict["shopify_access"],
    "Content-Type": "application/json",
    }
    products_link = f"https://{post_dict['shop_name']}.myshopify.com/admin/api/2024-01/products.json"

    response1 = requests.get(products_link, headers=shopify_headers)
    print('got all items')
    id_list=[]
    for product in response1.json()['products']:
        if product['title'] in created:
            id_list.append(product['id'])
    print('filtered list for just this collection')
    return id_list

In [70]:
def create_collection(created,week,year,post_dict):
    '''
    give this a list of product names (created) and it will add them to a collection for the week

    add something for the whole has the game happened yet aspect
    '''
    shopify_headers ={
    "X-Shopify-Access-Token": post_dict["shopify_access"],
    "Content-Type": "application/json",
    }
    collection_link = f"https://{post_dict['shop_name']}.myshopify.com/admin/api/2023-04/custom_collections.json"

    id_list = get_items(created, post_dict)
    collects = [{"product_id":x} for x in id_list]
    collection_data = {
    "custom_collection":{
        "title":f"Week {week}, {year}",
        "collects":collects
        }
    }
    response1 = requests.post(collection_link, headers=shopify_headers, json = collection_data)
    if response1.status_code==201:
        print("collection created")
    else:
        print("could not create collection")
        print(json.loads(response1.text))

In [71]:
create_collection([shop_config['title']],1,2023,shop_config)

got all items
filtered list for just this collection
collection created
